Step for running:
1. importing necessary libraries
2. Creating S3 bucket
3. Mapping train and test data in S3
4. Mapping the path of the models in S3

In [6]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [7]:
bucket_name = 'handsonmlops'
my_region = boto3.session.Session().region_name
print(my_region)

us-east-1


In [9]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ', e)

S3 bucket created successfully


In [10]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name,prefix)
print(output_path)

s3://handsonmlops/xgboost-as-a-built-in-algo/output


In [11]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [12]:
#Train test split 
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/opt/conda/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [15]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
from sagemaker.inputs import TrainingInput
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [17]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

Build Models Xgboost

In [24]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
# from sagemaker import image_uris
# container = image_uris.retrieve(boto3.Session().region_name,
#                           'xgboost', 
#                           repo_version='1.0-1')

In [23]:
from sagemaker import image_uris
import boto3

# Get the AWS region
region = boto3.Session().region_name

# Retrieve the image URI for XGBoost
container = image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1"  # Replace with your desired version
)

In [29]:
from sagemaker import estimator

# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }
     

# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)


In [30]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-01-10-07-49-31-553


2025-01-10 07:49:33 Starting - Starting the training job...
2025-01-10 07:49:48 Starting - Preparing the instances for training...
2025-01-10 07:50:31 Downloading - Downloading the training image......
2025-01-10 07:51:22 Training - Training image download completed. Training in progress...[2025-01-10 07:51:46.195 ip-10-0-231-115.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node trainin

Deploy Machine Learning model

In [33]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-01-10-09-16-22-706
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2025-01-10-09-16-22-706
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2025-01-10-09-16-22-706


-------!

In [37]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values
xgb_predictor.content_type = 'text/csv'  # set the data type for inference
xgb_predictor.serializer = CSVSerializer()  # khởi tạo đối tượng CSVSerializer
predictions = xgb_predictor.predict(test_data_array)  # Dự đoán trực tiếp
predictions = predictions.decode('utf-8')
predictions_array = np.fromstring(predictions[1:], sep=',')  # Cắt bỏ ký tự đầu tiên nếu có
print(predictions_array.shape)

(12357,)


In [38]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [39]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



In [40]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2025-01-10-09-16-22-706


[{'ResponseMetadata': {'RequestId': 'JWW2NPRS18YM4NQN',
   'HostId': 'VmzK5f5juYRMWAsStVj3ucjVm+cOTmEwcvkVoHY53LRTJOOhEYTF+CLJGzJ/KN3vrng6UhZrpMweD6ft/vka/tNjV/1CkwHZ',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'VmzK5f5juYRMWAsStVj3ucjVm+cOTmEwcvkVoHY53LRTJOOhEYTF+CLJGzJ/KN3vrng6UhZrpMweD6ft/vka/tNjV/1CkwHZ',
    'x-amz-request-id': 'JWW2NPRS18YM4NQN',
    'date': 'Fri, 10 Jan 2025 09:39:18 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-01-10-07-49-31-553/debug-output/events/000000000010/000000000010_worker_0.tfevents'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-01-10-07-49-31-553/debug-output/collections/000000000/worker_0_collections.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-01-10-07-49-31-553/debug-output/events/0000